In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

**Data Wrangling**

In [ ]:
'''list of integer list'''
def list_lines(file_name):
    with open(file_name) as df:
        df_line = df.read().splitlines()
    line_list = [[int(n) for n in line.split()] for line in df_line]
    return line_list


'''list of parameters'''
def get_params(line_list):
    top_line = line_list[0]
    params = {
        'drone_count' : top_line[2],
        'max_weight' : top_line[3],
        'max_time' : top_line[4]
    }
    return params


'''inserting the line b/w warehouse and cusotomer order '''
def get_wh_endline(line_list):
    wh_endline = 24 #line where warehouse data ends
    return wh_endline


'''weight of each product'''
def get_weights(line_list):
    weights = np.array(line_list[2])
    return weights.astype('int')


'''returns 2-D array of products by warehouses'''
def get_product_invent(line_list):
    wh_endline = get_wh_endline(line_list)
    supply = np.array(line_list[5:wh_endline:2])
    return supply


'''returns 2-D array of products by orders & array of total orders'''
def get_orders(line_list):
    wh_endline = get_wh_endline(line_list)
    orders = np.array(line_list[wh_endline+3::3])
    total_orders = np.array(line_list[wh_endline+2::3])
    return orders, total_orders


'''get warehouse and customer locations'''
def location(line_list):
    wh_endline = get_wh_endline(line_list)
    wh_loc = np.array(line_list[4:wh_endline:2])
    cust_loc = np.array(line_list[wh_endline+1::3])
    return wh_loc, cust_loc

In [ ]:
file = ['../input/hashcode-drone-delivery/busy_day.in']
line_list = list_lines(file[0])

params = get_params(line_list)
weights = get_weights(line_list)
supply = get_product_invent(line_list)
orders, total_orders = get_orders(line_list)
wh_loc, cust_loc = location(line_list)

***Creating dataframes for,***
1. Warehouse coordinates
2. Product data in each warehouse
3. Customer orders and coordinates

In [ ]:
'''Warehouse coordinates'''
wh_xaxis = [x[0] for x in wh_loc]
wh_yaxis = [x[1] for x in wh_loc]

warehouse_df = pd.DataFrame({'warehouse_x_axis': wh_xaxis, 'warehouse_y_axis': wh_yaxis})
warehouse_df

In [ ]:
'''Product data in each warehouse'''
product_df = pd.DataFrame(x for x in supply).transpose()
product_df.columns = [f'warehouse_{i}' for i in range(len(warehouse_df))]
product_df['Product_weight'] = weights

product_df.head()

In [ ]:
'''Customer orders and coordinates'''
cust_xaxis = [x[0] for x in cust_loc]
cust_yaxis = [x[1] for x in cust_loc]

max_len_order=max([len(x) for x in orders])

cust_df = pd.DataFrame(x for x in orders).fillna(0)
cust_df.columns = [f'Product_{i}_type' for i in range(max_len_order)]
cust_df["Total_orders"] = total_orders
cust_df["Order_x_coor"] = cust_xaxis
cust_df["Order_y_coor"] = cust_yaxis
cust_df.head()

In [ ]:
fig = plt.figure(figsize=(15,8))
data = fig.add_subplot()
data.scatter(wh_xaxis, wh_yaxis, c='b', s=50, label='Warehouse')
data.scatter(x = cust_xaxis, y = cust_yaxis, c='r', s=10, marker='x', label = 'Order_location')
plt.legend(loc='best')